In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.layers import concatenate, Dropout, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     #brightness_range=[0.75,1.25], 
                                     shear_range=10)

In [4]:
x1 = train.drop(['id', 'digit', 'letter'], axis=1).values
x1 = x1.reshape(-1, 28, 28, 1)
x1 = x1/255
x1_total = x1.copy()

def augment(x):
    aug_list = []
    for i in range(x1.shape[0]):
        num_aug = 0
        tmp = x1[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 9
for i in range(n):
    arr = augment(x1)
    x1_total = np.concatenate((x1_total, arr), axis=0)
    if i > n:
        break

print(x1_total.shape)

(20480, 28, 28, 1)


In [5]:
y1_data = train['digit']
y1 = np.zeros((len(y1_data), len(y1_data.unique())))
for i, digit in enumerate(y1_data):
    y1[i, digit] = 1

y1_total = y1.copy()
for i in range(n):
    arr = y1.copy()
    y1_total = np.concatenate((y1_total, arr), axis=0)

print(y1_total.shape)

(20480, 10)


In [6]:
x1_let = train['letter'].values
x1_let = x1_let[:, np.newaxis]
en = OneHotEncoder()
x1_let = en.fit_transform(x1_let).toarray()

x1_letter_total = x1_let.copy()
for i in range(n):
    arr = x1_let.copy()
    x1_letter_total = np.concatenate((x1_letter_total, arr), axis=0)

print(x1_letter_total.shape)

(20480, 26)


In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x1_total, y1_total, test_size=0.2, shuffle=True, stratify=y1_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x1_letter_total[:x1_train.shape[0],:]
x1_letter_val = x1_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

(16384, 28, 28, 1)
(4096, 28, 28, 1)
(16384, 10)
(4096, 10)
(16384, 26)
(4096, 26)


In [8]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
______________________________________________________________________________________________

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

cp = ModelCheckpoint('./models/{epoch:02d}-{val_accuracy:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x1_train, x1_letter_train], y1_train,
                    validation_data=([x1_val, x1_letter_val], y1_val), 
                    batch_size=64, epochs=50, verbose=1, callbacks = [cp])

Train on 16384 samples, validate on 4096 samples
Epoch 1/50
16384/16384 [==============================] - 25s 2ms/sample - loss: 1.0214 - accuracy: 0.6430 - val_loss: 1.2390 - val_accuracy: 0.5999
Epoch 2/50
16384/16384 [==============================] - 25s 2ms/sample - loss: 1.0099 - accuracy: 0.6552 - val_loss: 1.0444 - val_accuracy: 0.6836
Epoch 3/50
16384/16384 [==============================] - 29s 2ms/sample - loss: 0.8641 - accuracy: 0.7012 - val_loss: 0.9332 - val_accuracy: 0.7400
Epoch 4/50
16384/16384 [==============================] - 27s 2ms/sample - loss: 0.7614 - accuracy: 0.7382 - val_loss: 0.8233 - val_accuracy: 0.7632
Epoch 5/50
16384/16384 [==============================] - 29s 2ms/sample - loss: 0.6758 - accuracy: 0.7686 - val_loss: 0.7440 - val_accuracy: 0.7964
Epoch 6/50
16384/16384 [==============================] - 28s 2ms/sample - loss: 0.6056 - accuracy: 0.7919 - val_loss: 0.6232 - val_accuracy: 0.8086
Epoch 7/50
16384/16384 [==============================] -

In [14]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/45-0.9441.h5')

In [15]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255

x_letter_test = test['letter'].values
x_letter_test = x_letter_test[:, np.newaxis]
en = OneHotEncoder()
x_letter_test = en.fit_transform(x_letter_test).toarray()

In [16]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)

In [17]:
submission.to_csv('data/submission(CNN).csv', index=False)